In [1]:
import numpy as np
import matplotlib.pyplot as plt
import astropy.units as u
import warnings
from astropy.wcs import WCS
from astropy.io import fits
from scipy.optimize import curve_fit, root_scalar
from spectral_cube import SpectralCube
from lmfit import Model, Parameters
from scipy.interpolate import interp1d
import aplpy

In [2]:
warnings.filterwarnings('ignore')

# Constants
etamb = 0.89
etaf = 1.0
nu11 = 23.6944955e9
nu22 = 23.7226333e9
h = 6.62606896e-34
kB = 1.3806504e-23
Tbg = 2.73
c = 2.99792458e8
To = 41.5
epsilon = 8.854187817e-12
dipole = 298117.06e6
C_moment = 186726.36e6
mu = 1.476 * 3.336e-30
mu11 = mu ** 2 * (1. / 2.)
Einstein_A = (16. * np.pi ** 3 / (3. * epsilon * h * c ** 3)) * nu11 ** 3 * abs(mu11)
J_Tbg = h * nu11 / kB * (1. / (np.exp(h * nu11 / (kB * Tbg)) - 1))
etac = etamb * etaf
mNH3=17.03      #amu
mH=1.00794      #amu
amu=1.66053886e-27  #kg

In [3]:
def gaussian(x, amp, cen, wid):
    return amp * np.exp(-(x - cen)**2 / (2 * wid**2))

def quad_func(x, amp0, vel0, sigma0, tau0): # (1,1) Single-gaussian profile
    amp = [0.226, 0.273, 1.0, 0.277, 0.219] # Relative expected amplitudes of five-finger structure
    vel = [-19.503, -7.594, 0.0, 7.599, 19.493] # Relative expected velocities of five-finger structure
    emission_profile = np.zeros_like(x)
    for ampi, veli in zip(amp, vel):
        Tq=(1-np.exp(-ampi*tau0))/(1-np.exp(-tau0)) # Amplitude of gaussian is ratio of main/sattelite line amplitudes
        emission_profile += gaussian(x, Tq, vel0 + veli, sigma0)
    return amp0*emission_profile

def quad_func2c(x, amp0, vel0, sigma0, tau0, amp1, vel1, sigma1, tau1): # (1,1) Double-gaussian
    amp = [0.226, 0.273, 1.0, 0.277, 0.219]
    vel = [-19.503, -7.594, 0.0, 7.599, 19.493]

    emission_profile0 = np.zeros_like(x)
    emission_profile1 = np.zeros_like(x)
    for ampi, veli in zip(amp, vel):
        Tq0=(1-np.exp(-ampi*tau0))/(1-np.exp(-tau0))
        Tq1=(1-np.exp(-ampi*tau1))/(1-np.exp(-tau1))
        emission_profile0 += gaussian(x, Tq0, vel0 + veli, sigma0)
        emission_profile1 += gaussian(x, Tq1, vel1 + veli, sigma1)
    return amp0*emission_profile0+amp1*emission_profile1

def quad_func22(x, amp0, vel0, sigma0, tau0): # (2,2) Single-gaussian profile
    amp = [0.063, 0.039, 1.0, 0.092, 0.063]
    vel = [-26.023, -16.368, 0.0, 16.368, 26.023]

    emission_profile = np.zeros_like(x)
    for ampi, veli in zip(amp, vel):
        Tq=(1-np.exp(-ampi*tau0))/(1-np.exp(-tau0))
        emission_profile += gaussian(x, Tq, vel0 + veli, sigma0)
    return amp0*emission_profile

def quad_func222c(x, amp0, vel0, sigma0, tau0, amp1, vel1, sigma1, tau1): # (2,2) Double-gaussian profile
    amp = [0.063, 0.039, 1.0, 0.092, 0.063]
    vel = [-26.023, -16.368, 0.0, 16.368, 26.023]

    emission_profile0 = np.zeros_like(x)
    emission_profile1 = np.zeros_like(x)
    for ampi, veli in zip(amp, vel):
        Tq0=(1-np.exp(-ampi*tau0))/(1-np.exp(-tau0))
        Tq1=(1-np.exp(-ampi*tau1))/(1-np.exp(-tau1))
        emission_profile0 += gaussian(x, Tq0, vel0 + veli, sigma0)
        emission_profile1 += gaussian(x, Tq1, vel1 + veli, sigma1)
    return amp0*emission_profile0+amp1*emission_profile1

def solve_Tk(Tr): # Solve for kinetic temperature Tk given rotational temperature Tr
    if np.isnan(Tr):
        return np.nan

    def KineticTemp(Tk):
        if Tk <= 0:
            return 1e6  # prevent log or exp issues
        return Tk / (1. + ((Tk / To) * np.log(1. + (0.6 * np.exp(-15.7 / Tk))))) - Tr

    try:
        sol = root_scalar(KineticTemp, bracket=[1, 500], method='brentq')
        return sol.root if sol.converged else np.nan
    except Exception:
        return np.nan

In [ ]:
# File paths
# path = '/home/scratch/lmorgan/Projects/GalacticBar/'
path = '/users/hfwest/GBO-REU/'
# datadir = path+'Data/Pilot_All/Pilot_All/FitResults/'
datadir = path + 'RAMPS-Results/'
suffixout='_test'
Cube11 = SpectralCube.read(path + 'Data/Pilot_NH3_11_bl2.fits')
Cube22 = SpectralCube.read(path + 'Data/Pilot_NH3_22_bl2.fits')
NComp11_File = path + 'Data/NCompMap11.fits'
NComp11_hdu = fits.open(NComp11_File)[0]
NComp11_wcs = WCS(NComp11_hdu.header)
NComp11_data = NComp11_hdu.data
NComp22_File = path + 'Data/NCompMap22.fits'
NComp22_hdu = fits.open(NComp22_File)[0]
NComp22_wcs = WCS(NComp22_hdu.header)
NComp22_data = NComp22_hdu.data